# Lab 7: Wordle using OOP

In today's lab, we'll revisit our implementation of Wordle from week 5. If you don't have your own solution at hand, [you can download a sample solution](sources/lab07/wordle_solution.py). Of course, we still need [the list of accepted words, which you can find in this file](sources/lab07/word_list.txt) to play the game. Download these files and test if everything works. If you don't remember how Wordle was solved, take a quick look at the code.

In the first part of the lab, we will try to modify our solution to use object-oriented programming, so instead of functions, we will divide our code into separate modules, i.e. **classes**, with each class modeling one important element of the overall solution. Quickly review the theoretical knowledge regarding classes and objects, and then suggest a few classes that we could use to represent different components of the game.

The list of features we implemented two weeks ago may help you in your design. Try to think about what distribution of functionality would make sense for an object-oriented solution:

* `load_words(dataset_path)` - loads the list of allowed words from a file, the path to which it receives as an argument;
* `get_puzzle(word_list)` - returns a random element from the `word_list` that serves as the target word;
* `is_game_finished(guess, puzzle)` - checks if the user has already guessed the target word;
* `evaluate_guess(guess, puzzle)` - evaluates the user's guess and returns information about each letter in the word;
* `start_game(dataset_path)` - simulates a single game with a human player entering his/her guesses via the console (standard input).

## 1. Object-based solution

After the design, you may have identified two main components, namely the target word (`Puzzle`) and the game itself (`Game`). Since we only need input from the player, we don't need a special class to represent the player yet.

Try transforming your procedural solution into object-oriented code, with the following distribution of functionality:

* `Puzzle`:

    * `is_game_finished(self, guess)`
    * `evaluate_guess(self, guess)`

* `Game`:
    * `load_words(self, dataset_path)`
    * `generate_puzzle(self)` - similar to `get_puzzle()`, but without a return value, just creates an object of type `Puzzle`
    * `start_game(self)`

[The skeleton of the solution can also be found as a python script.](sources/lab07/wordle_oop.py)

**Note:** If you have designed a different solution, feel free to implement the functionality in a different style, you can compare the different solutions at the end.

In [ ]:
import random


class Puzzle:
    def __init__(self, word):
        self.puzzle = word.lower()

    def is_game_finished(self, guess):
        return False

    def evaluate_guess(self, guess):
        return list()


class Game:
    def __init__(self, dataset_path):
        self.word_list = self.load_words(dataset_path)
        self.puzzle = None

    def load_words(self, dataset_path):
        return list()

    def generate_puzzle(self):
        pass

    def start_game(self):
        pass

After the functionality has been implemented, you can test your solution with the following block of code:

In [ ]:
wordle = Game("word_list.txt")
wordle.start_game()

## 2. Bots on set

It's a known fact that programmers are lazy, and although Wordle is a fun game and a great way to procrastinate, sooner or later they will think of implementing a smart bot to play the game for them much more efficiently. In this task, we will create one such bot - through the definition of another class, of course.

In [ ]:
from copy import deepcopy
import string


PLAYER_KNOWLEDGE = [(letter, None, -1)
                    for letter in string.ascii_lowercase]


class Bot:
    def __init__(self, word_list):
        self.reset(word_list)

    def reset(self, word_list):
        pass

    def get_player_guess(self):
        return ""

    def process_result(self, result):
        pass

### 2.1. The bot is getting ready

As you can see, unlike the previous classes, no value is set directly in the `Bot` constructor, instead only the `reset()` method is called to prepare the bot for solving the problem. Our bot will be given two values, which you must initialize in this method:

* `self.knowledge` - a representation of what the bot knows about the search word (updated based on feedback from the game). This will be a list of tuples in the format set by `PLAYER_KNOWLEDGE` (structure described below).
* `self.possibles` - a list of words that are accepted by the game (as output of `load_words()`).

The bot's knowledge will be represented in the `knowledge` list, to be initialized as a copy of the `PLAYER_KNOWLEDGE` constant (a deep copy, since it is a list of objects/tuples). This list is initialized at the beginning of the game and is a list of tuples, where each triplet represents information about each possible letter with the following structure:

 - letter - string of length 1
 - information about whether the letter is in the word - initialized to `None`, later you replace the value with boolean values `True/False`
 - information about the position of the letter in the correct solution - initialized to -1, later replaced by a valid index of 0 to 4.
 
**Note:** This bot will not account for the possibility of multiple occurrences of a letter in a word, so the third value will always be a single number.

**Task:** Implement the `reset()` method to initialize the member variables of the class as copies of the constant or parameter.

### 2.2. The bot guesses

The next step is to implement the `get_player_guess()` method, which first selects a list of possible solutions from the `self.possibles` word list, and returns a random guess. The bot eliminates possibilities based on the knowledge of the correct solution available in `self.knowledge`.

Based on this knowledge, the `get_player_guess` method should eliminate possibilities as follows:
 1. delete from the list of possible solutions words that do not contain letters it knows are in the correct solution based on the `self.knowledge` list
 2. delete from the list of possible solutions words that contain letters that it knows are not in the correct solution based on the `self.knowledge` list
 3. delete from the list of possible solutions words that do not have the correct letter in some position if it already knows what letter will be there.

For simplicity, we will also give an example. Suppose the bot knows that the solution definitely contains the letters *r*, *s*, *e*, *u*; definitely does not contain the letter *a*; and the first position has the letter *r*. Thus, based on the individual rules, it would eliminate, for example:
 1. the word *table*, which does not contain the necessary letters *r*, *s*, *u*
 2. the word *braid*, which contains the letter *a*, which the player already knows is not in the solution
 3. the word *trout*, since the player already knows that the letter *r* must be in the first position

After eliminating some possible solutions, the bot chooses a random word from the remaining list. Be sure to update the member variable `self.possibles` appropriately before returning a random guess.

**Task:** Implement the `get_player_guess()` method according to the above specification.

### 2.3. The bot learns (?)

Of course, the bot has a chance to win the game only if it gradually updates its knowledge of the target word. This is done by using the `process_result()` method with the `result` parameter, which contains the feedback from the game that is generated by the `evaluate_guess()` method (the format does not change). The method sequentially processes information about all the letters from the last guess as follows:
 1. updates the information about whether or not the letter is in the correct solution
 2. if the letter is also in the correct position, it updates this information in the `self.knowledge` list.

The `process_result()` method has no return value, update the `self.knowledge` list directly.

**Task:** Implement the `process_result(result)` method as described above, and this completes the implementation of your bot.

**Note:** Remember that the `self.knowledge` list contains tuples, which are immutable. This is why when processing feedback, you need to create a new tuple and store it in the correct place in the knowledge list. The order of the information about the letters in the list should remain the same, i.e. you will first have knowledge about the occurrences of the letter *a*, the letter *b*, etc.

## 3. Completing the game

After implementing the `Bot` class, we have the bot ready, but we can't test its functionality yet, since our game doesn't support interaction with the bot.

**Task:** Add the method `bot_game(bot)` to the `Game` class, which receives as a parameter the bot object that wants to play the game. The method is very similar to `start_game()` - but the input from the player is replaced by the input from the bot. During the game, you can also output a list of possible solutions according to the bot, which will get progressively shorter as the bot processes the information it learns about the search word.

You can then test your solution using the code:

In [ ]:
wordle = Game("word_list.txt")
agent = Bot(wordle.word_list)
wordle.bot_game(agent)